In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.0 MB 1.2 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 22.12.6 which is incompatible.


In [2]:
import mediapipe as mp
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import pandas as pd
import openpyxl
import os
#mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic



In [3]:
video_folder_path = '/content/drive/MyDrive/videos'
excel_folder_path = '/content/drive/MyDrive/xls'
if not os.path.exists(excel_folder_path):
  os.makedirs(excel_folder_path)
files = os.listdir(video_folder_path)


In [21]:
# Initiate holistic model
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
for item in files:
  cap = cv2.VideoCapture(video_folder_path + '/' + item)
  counter = 0
  df = pd.DataFrame(columns= ['frame', 'x', 'y', 'z'])
  while cap.isOpened():
    ret, frame = cap.read()
    
    if ret == True:
      counter += 1
      # Convert the feed from BGR to RGB
      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      # Refer the section above how to make detections on feed
      results = holistic.process(image)

      # saving to excel
      data = [{'frame': counter, 'x': i.x,'y': i.y,'z': i.z} for i in results.pose_landmarks.landmark]
      df1 = pd.DataFrame(data = data)
      df = df.append(df1)

      if cv2.waitKey(5) & 0xFF == 27:
        break

    else:
      break
  name = excel_folder_path + '/' + item[:-3] + 'xlsx'
  df = df.astype(float)
  df.to_excel(name)

  cap.release()
  cv2.destroyAllWindows()

In [5]:
connections = [i for i in mp_holistic.POSE_CONNECTIONS]
print(connections)

[(15, 21), (16, 20), (18, 20), (3, 7), (14, 16), (23, 25), (28, 30), (11, 23), (27, 31), (6, 8), (15, 17), (24, 26), (16, 22), (4, 5), (5, 6), (29, 31), (12, 24), (23, 24), (0, 1), (9, 10), (1, 2), (0, 4), (11, 13), (30, 32), (28, 32), (15, 19), (16, 18), (25, 27), (26, 28), (12, 14), (17, 19), (2, 3), (11, 12), (27, 29), (13, 15)]


In [ ]:
excel_files = os.listdir(excel_folder_path)
res=(500,500) #resulotion
pose_folder_path = '/content/drive/MyDrive/pose'
if not os.path.exists(pose_folder_path):
  os.makedirs(pose_folder_path)

for item in excel_files:
  name = pose_folder_path + '/' + item[:-4] + 'avi'
  out = cv2.VideoWriter(name,cv2.VideoWriter_fourcc(*'DIVX'), 60, res)

  df = pd.read_excel(excel_folder_path + '/' +item)

  counter = 1
  lmk = df.loc[(df['frame']==counter)]
  img_arr = []
  while not lmk.empty:
    points = [(int(lmk['x'][i]*res[0]),int(lmk['y'][i]*res[1])) for i in lmk.index]
    img = np.ones(shape=(res[0],res[1],3), dtype=np.uint8)

    for i,j in points:
      cv2.circle(img,(i,j), 3, (0,0,255), 0)

    for i,j in connections:
      cv2.line(img,points[i], points[j],(255,0,0), 1)

    img_arr.append(img) 
    counter += 1
    lmk = df.loc[(df['frame']==counter)]

  for i in range(len(img_arr)):
    out.write(img_arr[i])
    
  out.release()

